# Tutorial 2: Interacting with images in phenopype

In this tutorial we learn how to open and close images in phenopype, and how to use the interactive featues of the program.  Phenopype uses OpenCV's [HighGUI module](https://docs.opencv.org/3.4/d7/dfc/group__highgui.html) to display images and to allow users to interact with images. HighGUI has a few pros and cons: 

```
+ native OpenCV GUI (no extra GUI libraries required)
+ the module is extremely fast in displaying images 
+ it can display very large image-arrays (> 10000x10000 pixels)
+ it can display multiple images side by side
+ interactions (drawing and measuring) are possible 

- sometimes unstable (e.g. windows are not closed but freeze)
- issues with cross plattform stability (e.g. on macOS)
- displaying instructions is hacky (text is "painted" onto a displayed image) 
- user input (key strokes and mouse clicks) sometimes isn't captured properly
```

Currently Phenopype uses the standard HighGUI libraries that ship with the most recent precompiled `opencv-contrib-python` package that is listed on the [Python package index](https://pypi.org/project/opencv-python/), which is `Qt` for Linux and macOS, and `Win32 UI` on Windows. The `Qt` GUI is a bit more userfriendly with builtin buttons, scrollbars, RGB info and zoom ([see the OpenCV docs](https://docs.opencv.org/master/dc/d46/group__highgui__qt.html)), but you don't actually need those things for basic Phenopype GUI interactions.

## Window control

**IMPORTANT - READ BEFORE RUNNING THE CODE** 

### Open

In a very simple case, where you want to just inspect an image file from within phenopype, you use the `load_image` function to load an image file into Python as an array, and then display it with `show_image`. A HighGUI window will pop up and display the array. While the window is open, the Python kernel is "busy", and you cannot interact with the console, i.e. run any code - you first have to close the image again.

### Close
 
Although the HighGUI window has the "red crossed" close button in the upper right corner, DO NOT USE IT! For practical reasons,  phenopype relies on key strokes to control the windows. Make sure that the window is selected / highlighted, and use the following key combinations to close it:  

`Enter` - close a window
`Ctrl+Enter` - close and finish a window in `pype`-mode 
`Esc` - close a window and quit the Phenoype process that invoked it. This may also work when the process is frozen. 

### Issues

- If a keystroke doen't do anything the first time, try a few times more (Phenopype "listens" to your keystroke while refreshing the presented image, and sometimes a refreshing operation overlaps with user input and it is not recognized).
- If your keystore still isn't recognized, make sure the window is highlighted (i.e. click on it) and try again.
- If you killed a process but the window still open (i.e. the kernel is not busy anymore), type 
`import cv2` and `cv2.destroyAllWindows()` into the console to close the window.
- If a window and the Python kernel is frozen permanently, you need to restart it - sorry!

## Opening images in phenopype

To open an image, it first has to be loaded as an array using `load_image`. The array can then be passed on to `show_image`, which simply displays an image (no interactions, except zooming in using the mousewheel). The window is closed by keystroke (`Enter`, or `Esc` which both closes the window and ends ongoing processes). 

In [1]:
import phenopype as pp
import os

img = pp.load_image(os.path.join("images",'isopods.jpg')) 
pp.show_image(img)

`show_image` can also handle multiple images. Here we loop through the *images* folder, attach all images to a list, and pass that list of arrays to the functions - it will give a warning if more than 10 images are being opened at the same time. 

In [ ]:
import os
images = [] # square-brakets make an empty list
names = os.listdir("images") # making a list of all the files names inside a directory

for i in names: # looping along our list of names
    filepath = os.path.join("images", i) # joining name and path strings 
    images.append(pp.load_image(filepath)) # load images and store them in list

pp.show_image(images, position_offset=100) ## show all images in the image folder in the tutorial directory

The function has a few more options to arrange and separte images across the screen. Future version of phenopype will do this in more meaningful manner (e.g., arrange small images side by side until the screen is filled).  

In [ ]:
pp.show_image(images, 
              max_dim=250,           # maximum dimension (in either direction) for the windows
              check=False,           # don't issue warning if more than 10 images are opened
              position_offset=50,    # window offset if multiple windows are displayed
              position_reset=False)  # don't reset position of windows (i.e. window position will be remembered)

## Creating masks

Masking, i.e. removing unwanted parts of an image that contain noise by including or excluding certain parts of the image, is an important preprocessing step in any computer vision workflow. Phenopype's `create_mask` tool provides flexibility when drawing masks.   

<center>
<div style="width:500px; text-align: left" >
    
![Create masks](_assets/masks1.gif)
    
**Fig. 1:** Phenopype's mask tool in action. You can include or exclude certain parts of the image; the resulting coordinates are recognized in subsequent computer vision steps (e.g. thresholding)
    
</div>
</center>

Using `create_mask` results in a DataFrame object that contains coordinates of the created mask. You can add multiple "submasks" that belong to the same mask layer that don't have to be connected. Finish with `Enter`.

In [2]:
img = pp.load_image(os.path.join("images",'isopods.jpg')) 
masks = pp.preprocessing.create_mask(img, label="tray")
masks

- creating mask


,filename,coords,include,mask
0,unknown,"[(262, 153), (1879, 153), (1879, 1347), (262, ...",1.0,tray


The masks DataFrame can also be attached to an existing DataFrame, when provided via `df_image_data`, to concatenate information, e.g. to pass on the image file name. 

In [9]:
img, df = pp.load_image(os.path.join("images",'isopods.jpg'), df=True) 
masks = pp.preprocessing.create_mask(img, label="tray",df_image_data=df)
masks

- creating mask


,filename,width,height,coords,include,mask
0,isopods.jpg,2100.0,1400.0,"[(212, 174), (1917, 174), (1917, 1345), (212, ...",1.0,tray


Creating masks like this corresponds to the [prototyping workflow](tutorial_3_phenopype_workflow.ipynb#Prototyping-worflow), which requires an explicit call of `draw_masks` to visualize the mask that was just created. A "canvas" is created with the original image and the mask coordinates, which can then be shown using `show_image`:

In [10]:
canvas = pp.visualization.draw_masks(img, 
                                     df_masks=masks, 
                                     label=True, 
                                     label_colour="black", 
                                     label_size=3, 
                                     line_width=2,
                                     label_width=4)    
pp.show_image(canvas)

drawing mask: tray


Existing mask DataFrames can be updated if they are provided via `df_mask`. In this step, select the reference card to and set `include=False` to exclude it. We use the previously created canvas to know which areas have been masked already. Then we create and visualize a new canvas, which will draw both masks onto the original image. Note that the masks with `include=False` are drawn in red.

In [12]:
masks = pp.preprocessing.create_mask(canvas, 
                                     df_masks=masks, 
                                     label="scale", 
                                     include=False # will be automatically drawn in red
                                    )
canvas = pp.visualization.draw_masks(img, 
                                     df_masks=masks, 
                                     label=True, 
                                     label_colour="black", 
                                     label_size=3, 
                                     line_width=2,
                                     label_width=4)    
pp.show_image(canvas)

- mask with label scale already created (edit/overwrite=False)
drawing mask: tray
drawing mask: scale


If you need more flexibility while drawin a mask, try the `polygon` tool - single polygons are finished with `Ctrl`, finish with `Enter` - the last open polyon will automatically be completed.

In [13]:
masks = pp.preprocessing.create_mask(img, df_masks=masks, tool="polygon")
canvas = pp.visualization.draw_masks(img, df_masks=masks)    
pp.show_image(canvas)

drawing mask: tray
drawing mask: scale
drawing mask: mask1


## Landmarks, lines, and drawings

Functional morphology of organisms is often measured by placing landmarks at specific points that show structural, functional or developmental significance. In Phenopype, this is done using the `landmark` tool. 

<center>
<div style="text-align: left">
    
![Landmarks in high throughput](_assets/ex2_ht.gif)
    
**Fig. 2:** Phenopype's landmark tool (here called with the high throughput method). 
</div>
</center>

Load the image, and place landmarks. Finish with `Enter`.

In [ ]:
img = pp.load_image(os.path.join("images",'stickle1.jpg')) 
df_lm = pp.measurement.landmarks(img)

There are a few options to customize the landmarks ...

In [ ]:
df_lm = pp.measurement.landmarks(img, 
                                 point_size=15, 
                                 point_colour="red", 
                                 label_size=2,
                                 label_width=3, 
                                 label_colour="blue") 

... and, just like for the masks, we need to explicitly call a drawing function. These options can be different from the actual `landmark` function.

In [ ]:
canvas = pp.visualization.draw_landmarks(img, df_landmarks=df_lm,
                                         point_size=15, 
                                         point_colour="green", 
                                         label_size=2,
                                         label_width=3, 
                                         label_colour="blue")    
pp.show_image(canvas)

Similiar to the `polygon` tool in the `create_mask` function, one can use the `draw_polyline` tool to measure an object. Try to measure the length of the lateral area on the stickleback is plated. 

In [ ]:
df_lines = pp.measurement.polylines(img, 
                                 line_width=2, 
                                 line_colour="green") 
canvas = pp.visualization.draw_polylines(canvas, df_polylines=df_lines,
                                         line_width=2, 
                                         line_colour="green")    
pp.show_image(canvas)
df_lines

Similar to this function is the `draw` tool, which is useful to separate objects after binarization. Here, for example, use a combination of `create_mask` and `draw` to detect and separate connected stickleback plates: first draw a mask, then do the segmentation. The `draw` tool is used directly on the binary image, and always creates lines with two points (or rectangles when `tool="rectangle"` is selected. Then find the contours, and show the result.

In [14]:
image = pp.load_image(os.path.join("images",'stickle1.jpg')) 
mask = pp.preprocessing.create_mask(image, tool="polygon")

- creating mask


In [15]:
image_bin = pp.segmentation.threshold(image, 
                                      method="adaptive", 
                                      channel="red",
                                      blocksize=199, 
                                      constant=5,
                                      df_masks=mask) 
image_morph = pp.segmentation.morphology(image_bin, 
                                         operation="open", 
                                         shape="ellipse",
                                         kernel_size=3, 
                                         iterations=1)

- including pixels from 1 drawn masks 


In [18]:
image_bin_draw, drawing_coords = pp.segmentation.draw(image_morph)

- drawing


In [19]:
contours = pp.segmentation.find_contours(image_bin_draw,  retrieval="ext", min_area=150)
image_drawn = pp.visualization.draw_contours(image, df_contours=contours) 
pp.show_image(image_drawn)

- found 11 contours that match criteria


## Data entry

When phenopype windows are openened they can also receive text input, for example, to type in data that is visible inside the image: often, the picture name contains the ID, but typically an label is placed inside the image. Using the `enter_data` tool we open the image and an entry prompt that will create a column with a name of our chosing inside all exported results.

In [20]:
image, df_img_data = pp.load_image(os.path.join("images",'stickle1.jpg'), df=True) 

In [21]:
df_img_data = pp.preprocessing.enter_data(image, df_img_data, columns="ID")

- add column ID


In [22]:
df_img_data

,filename,width,height,ID
0,stickle1.jpg,2400,1600,142501
